In [8]:
import torch
from torch import nn
from torch.utils import data
import torchvision
from torchvision import transforms
dropout1, dropout2 = 0.2, 0.5
net = nn.Sequential(nn.Flatten(),
        nn.Linear(784, 256),
        nn.ReLU(),
        # 在第一个全连接层之后添加一个dropout层
        nn.Dropout(dropout1),
        nn.Linear(256, 256),
        nn.ReLU(),
        # 在第二个全连接层之后添加一个dropout层
        nn.Dropout(dropout2),
        nn.Linear(256, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)
def load_data_fashion_mnist(batch_size, resize=None):  #@save
    """下载Fashion-MNIST数据集，然后将其加载到内存中"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(
        root=r"C:\Users\dmy\Desktop\visiondata", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(
        root=r"C:\Users\dmy\Desktop\visiondata", train=False, transform=trans, download=True)
    return (data.DataLoader(mnist_train, batch_size, shuffle=True,
                            num_workers=12),
            data.DataLoader(mnist_test, batch_size, shuffle=False,
                            num_workers=12))
def train_epoch_ch3(net, train_iter, loss):  #@save
    """训练模型一个迭代周期（定义见第3章）"""
    # 将模型设置为训练模式
    if isinstance(net, torch.nn.Module):
        net.train()
    # 训练损失总和、训练准确度总和、样本数
    metric = Accumulator(3)
    updater=torch.optim.SGD(net.parameters(), lr=0.1)
    #X,y=next(iter(train_iter))
    for X, y in train_iter:
        # 计算梯度并更新参数
        y_hat = net(X)
        l = loss(y_hat, y)
        updater.zero_grad()
        l.mean().backward()
        updater.step()
    # else:
    #     # 使用定制的优化器和损失函数
    #     l.sum().backward()
    #     updater(X.shape[0])
    metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())  # 返回训练损失和训练精度
    return metric[0] / metric[2], metric[1] / metric[2]
class Accumulator:  #@save
    """在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
def evaluate_accuracy(net, data_iter):  #@save
    """计算在指定数据集上模型的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval()  # 将模型设置为评估模式
    metric = Accumulator(2)  # 正确预测数、预测总数
    with torch.no_grad():
        for X, y in data_iter:
            #print("Input shape:", X.shape,y.shape)
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]
def accuracy(y_hat, y):  #@save
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y

    return (cmp.type(y.dtype).sum())
def train_ch3(net, train_iter, test_iter, loss, num_epochs):  #@save
    """训练模型（定义见第3章）"""

    for epoch in range(num_epochs):
        train_metrics = train_epoch_ch3(net, train_iter, loss)
        print(f"epoch:{epoch}",train_metrics[0],train_metrics[1])
        test_acc = evaluate_accuracy(net, test_iter)

    train_loss, train_acc = train_metrics
    assert train_loss < 0.5, train_loss
    assert train_acc <= 1 and train_acc > 0.7, train_acc
    assert test_acc <= 1 and test_acc > 0.7, test_acc


In [ ]:
num_epochs = 10
net.apply(init_weights)
num_epochs, lr, batch_size = 10, 0.5, 256
loss = nn.CrossEntropyLoss(reduction='none')
train_iter, test_iter = load_data_fashion_mnist(batch_size)

train_ch3(net, train_iter, test_iter, loss, num_epochs)

100.0%
100.0%
100.0%
100.0%


TypeError: train_ch3() takes 5 positional arguments but 6 were given